# Motor Vehicle Collisions crash table
Data from: https://catalog.data.gov/dataset/motor-vehicle-collisions-crashes


In [68]:
# Imports 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import random

In [69]:
data = pd.read_csv('Motor_Vehicle_Collisions_-_Crashes.csv')

/var/folders/3y/snyjd7qd59d_gxq1x0y5gywh0000gn/T/ipykernel_39538/2458010851.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('Motor_Vehicle_Collisions_-_Crashes.csv')


In [70]:
data.head()
data = data.drop_duplicates()


In [71]:
contributing_Factors_vehicle_1 = data["CONTRIBUTING FACTOR VEHICLE 1"].value_counts()
data['CONTRIBUTING FACTOR VEHICLE 2'].value_counts()

CONTRIBUTING FACTOR VEHICLE 2
Unspecified                       1528459
Driver Inattention/Distraction      97810
Other Vehicular                     33116
Following Too Closely               19977
Failure to Yield Right-of-Way       17630
                                   ...   
Eating or Drinking                     13
Listening/Using Headphones             12
Windshield Inadequate                   7
1                                       5
Texting                                 3
Name: count, Length: 61, dtype: int64

In [72]:
df = data.copy()

num_3_crashes = df.dropna(subset=['CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3'])
num_crashes_3_or_more = num_3_crashes.shape[0]
print(num_3_crashes.head())
print(f"Number of crashes with 3 or more contributing factors: {num_crashes_3_or_more}")

     CRASH DATE CRASH TIME   BOROUGH ZIP CODE   LATITUDE  LONGITUDE  \
2    11/01/2023       1:29  BROOKLYN  11230.0  40.621790 -73.970024   
25   12/14/2021      14:30       NaN      NaN  40.783268 -73.824850   
69   04/24/2022       0:30       NaN      NaN  40.638382 -74.036680   
119  03/10/2022       8:30  BROOKLYN  11238.0  40.676403 -73.962870   
121  03/26/2022       7:36  BROOKLYN  11209.0  40.613926 -74.030174   

                    LOCATION         ON STREET NAME CROSS STREET NAME  \
2     (40.62179, -73.970024)          OCEAN PARKWAY          AVENUE K   
25    (40.783268, -73.82485)  WHITESTONE EXPRESSWAY               NaN   
69    (40.638382, -74.03668)           BELT PARKWAY               NaN   
119   (40.676403, -73.96287)         PROSPECT PLACE      GRAND AVENUE   
121  (40.613926, -74.030174)  FORT HAMILTON PARKWAY         97 STREET   

    OFF STREET NAME  ...  CONTRIBUTING FACTOR VEHICLE 2  \
2               NaN  ...                    Unspecified   
25              

In [73]:
import streamlit as st
import pandas as pd 
import altair as alt
import plotly.express as px 


In [74]:
st.set_page_config(
    page_title="test", 
    page_icon="X",
    layout="wide", 
    initial_sidebar_state="expanded"
)
alt.themes.enable("dark")

2025-03-13 14:47:35.679 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


ThemeRegistry.enable('dark')

In [75]:
columns_to_drop = [
    'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 
    'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5', 
    'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2', 
    'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4', 
    'CONTRIBUTING FACTOR VEHICLE 5', 'NUMBER OF PERSONS INJURED', 
    'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED', 
    'NUMBER OF MOTORIST KILLED', 'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF PERSONS KILLED'
]
copy = data.copy()
copy = copy.drop(columns=columns_to_drop)
copy = copy.dropna(subset=['LATITUDE', 'LONGITUDE'])
copy.rename(columns={'LATITUDE': 'latitude', 'LONGITUDE': 'longitude'}, inplace=True)


In [76]:
copy['CRASH DATE'].min()
copy['CRASH DATE'] = pd.to_datetime(copy['CRASH DATE'])
filtered_data = copy[(copy['CRASH DATE'].dt.year >= 2020) & (copy['CRASH DATE'].dt.year <= 2025)]
year = copy['CRASH DATE'].min().year
print(year)

2012


In [77]:
# Define the bounding box for New York City
min_lat, max_lat = 40.477399, 40.917577
min_lon, max_lon = -74.259090, -73.700272

# Filter the data for accidents within the bounding box
nyc_data = copy[(copy['latitude'] >= min_lat) & (copy['latitude'] <= max_lat) & 
                (copy['longitude'] >= min_lon) & (copy['longitude'] <= max_lon)]

# Remove outliers by ensuring latitude and longitude are within the bounding box
nyc_data = nyc_data[(nyc_data['latitude'] != 0) & (nyc_data['longitude'] != 0)]

print(nyc_data.head())
print(f"Number of accidents in New York City area: {nyc_data.shape[0]}")

   CRASH DATE CRASH TIME   BOROUGH ZIP CODE   latitude  longitude  \
2  2023-11-01       1:29  BROOKLYN  11230.0  40.621790 -73.970024   
9  2021-09-11       9:35  BROOKLYN  11208.0  40.667202 -73.866500   
10 2021-12-14       8:13  BROOKLYN  11233.0  40.683304 -73.917274   
12 2021-12-14      17:05       NaN      NaN  40.709183 -73.956825   
13 2021-12-14       8:17     BRONX  10475.0  40.868160 -73.831480   

                   LOCATION              ON STREET NAME CROSS STREET NAME  \
2    (40.62179, -73.970024)               OCEAN PARKWAY          AVENUE K   
9     (40.667202, -73.8665)                         NaN               NaN   
10  (40.683304, -73.917274)             SARATOGA AVENUE    DECATUR STREET   
12  (40.709183, -73.956825)  BROOKLYN QUEENS EXPRESSWAY               NaN   
13    (40.86816, -73.83148)                         NaN               NaN   

                OFF STREET NAME  COLLISION_ID  
2                           NaN       4675373  
9       1211      LORING A

In [78]:
from geopy.geocoders import Nominatim

def fill_missing_borough_zipcode(data):
    geolocator = Nominatim(user_agent="mvc_streamlit")
    
    def get_location_info(row):
        if pd.isna(row['BOROUGH']) or pd.isna(row['ZIP CODE']):
            try:
                print(f"{row['latitude']}, {row['longitude']}")
                location = geolocator.reverse(f"{row['latitude']}, {row['longitude']}")
                
                print("Loc: " ,location)
                if location:
                    address = location.raw.get('address', {})
                    row['BOROUGH'] = address.get('borough', row['BOROUGH'])
                    row['ZIP CODE'] = address.get('postcode', row['ZIP CODE'])
            except Exception as e:
                print("Exception: ", e)
        return row

    data = data.apply(get_location_info, axis=1)
    return data

#nyc_data = fill_missing_borough_zipcode(nyc_data)


In [79]:
from sklearn.neighbors import NearestNeighbors
def fill_zip_boroughs_with_closest(data): 
    data = data.dropna(subset=['LATITUDE', 'LONGITUDE'])
    # Going to match the closest coordinates to with the zip code and maybe the street? 
    missing_borough_zip = data[pd.isna(data['BOROUGH']) | pd.isna(data['ZIP CODE'])]
    complete_borough_zip = data.dropna(subset=['BOROUGH', 'ZIP CODE'])
    coords = complete_borough_zip[['LATITUDE', 'LONGITUDE']]
    nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(coords)
    distances, indices = nbrs.kneighbors(missing_borough_zip[['LATITUDE', 'LONGITUDE']])

    # Use vectorized operations to update missing values
    missing_borough_zip.loc[:, 'BOROUGH'] = complete_borough_zip.iloc[indices.flatten()]['BOROUGH'].values
    missing_borough_zip.loc[:, 'ZIP CODE'] = complete_borough_zip.iloc[indices.flatten()]['ZIP CODE'].values

    data.update(missing_borough_zip)
    return data

In [80]:

print("Number of NaNs in BOROUGH before calling the function:", data['BOROUGH'].isna().sum())
data = fill_zip_boroughs_with_closest(data)
print("Number of NaNs in BOROUGH after calling the function:", data['BOROUGH'].isna().sum())
data.to_csv('filled_data.csv')


Number of NaNs in BOROUGH before calling the function: 667675
Number of NaNs in BOROUGH after calling the function: 0
